In [1]:
import pandas as pd
import numpy as np

In [5]:
INDEX = "MS50"

#Carico tutti i dati
prices = pd.read_csv( f"./data/{INDEX}/prices_D.csv" )
shorts = pd.read_csv( f"./data/{INDEX}/short_interest.csv" )
trends = pd.read_csv( f"./data/{INDEX}/trends_W.csv" )
news = pd.read_csv( f"./data/{INDEX}/news_volume_D.csv" )
wiki = pd.read_csv( f"./data/{INDEX}/wiki_views_D.csv" )

tickers = pd.read_csv( f"./data/{INDEX}/tickers.csv" )["ticker"].unique()
prices = prices[prices["ticker"].astype(str).isin([str(t) for t in tickers])].copy()

In [6]:
#prendo i prices alla data degli shorts
prices = prices.copy()
shorts = shorts.copy()

prices["ticker"] = prices["ticker"].astype(str)
shorts["ticker"] = shorts["ticker"].astype(str)

prices["date"] = pd.to_datetime(prices["date"], errors="coerce").dt.tz_localize(None)
shorts["date"] = pd.to_datetime(shorts["date"],  errors="coerce").dt.tz_localize(None)

prices = prices.dropna(subset=["ticker","date"])
shorts = shorts.dropna(subset=["ticker","date"])

prices = prices[prices["ticker"].isin([str(t) for t in tickers])].copy()
shorts = shorts[shorts["ticker"].isin([str(t) for t in tickers])].copy()

left = (shorts[["ticker","date"]]
        .drop_duplicates()
        .sort_values("date", kind="mergesort")           
        .reset_index(drop=True))

right = (prices[["ticker","date","close"]]
         .sort_values("date", kind="mergesort")             
         .reset_index(drop=True))

prices_on_short = pd.merge_asof(
    left=left,
    right=right,
    on="date",            
    by="ticker",               
    direction="backward",
    allow_exact_matches=True
)

In [11]:
#prendo i trends alla data degli shorts (il precendente o la media dei due precedenti)
shorts["date"] = pd.to_datetime(shorts["date"], errors="coerce").dt.tz_localize(None).dt.floor("D")
trends["date"] = pd.to_datetime(trends["date"], errors="coerce", utc=True).dt.tz_convert(None).dt.floor("D")

shorts = shorts.dropna(subset=["ticker","date"]).reset_index(drop=True)
trends = trends.dropna(subset=["ticker","date"]).reset_index(drop=True)

shorts = shorts.sort_values(["ticker","date"], kind="mergesort").reset_index(drop=True)
trends = trends.sort_values(["ticker","date"], kind="mergesort").reset_index(drop=True)

trends["prev_score"] = trends.groupby("ticker")["score"].shift(1)

left_curr = shorts[["ticker","date"]].sort_values("date", kind="mergesort").reset_index(drop=True)
right_curr = trends[["ticker","date","score"]].rename(columns={"date":"trend_date","score":"s1"}).sort_values("trend_date", kind="mergesort").reset_index(drop=True)

curr = pd.merge_asof(
    left=left_curr,
    right=right_curr,
    left_on="date",
    right_on="trend_date",
    by="ticker",
    direction="backward",
    allow_exact_matches=True
)

curr = curr.merge(
    trends[["ticker","date","prev_score"]].rename(columns={"date":"trend_date"}),
    on=["ticker","trend_date"],
    how="left"
)

curr["trend_score2w"] = np.where(
    curr["prev_score"].notna(),
    (curr["s1"] + curr["prev_score"]) / 2.0,
    curr["s1"]
)

trends_window = curr.drop(columns=["trend_date","s1","prev_score"]).sort_values(["ticker","date"], kind="mergesort").reset_index(drop=True)

In [9]:
#prendo il numero di news alla data degli shorts (calcolati dallo shorts precedente allo shorts attuale)

shorts["date"] = pd.to_datetime(shorts["date"], errors="coerce").dt.tz_localize(None).dt.floor("D")
news["date"]  = pd.to_datetime(news["date"],  errors="coerce", utc=True).dt.tz_convert(None).dt.floor("D")

shorts = shorts.dropna(subset=["ticker","date"]).reset_index(drop=True)
news  = news.dropna(subset=["ticker","date"]).reset_index(drop=True)

shorts = shorts.sort_values(["ticker","date"], kind="mergesort").reset_index(drop=True)
shorts["prev_date"] = shorts.groupby("ticker")["date"].shift(1)

news = news.sort_values(["ticker","date"], kind="mergesort").reset_index(drop=True)
news["cum_news"] = news.groupby("ticker")["news_count"].cumsum()

left_curr  = shorts[["ticker","date"]].sort_values("date", kind="mergesort").reset_index(drop=True)
right_curr = news[["ticker","date","cum_news"]].sort_values("date", kind="mergesort").reset_index(drop=True)

curr = pd.merge_asof(
    left=left_curr,
    right=right_curr,
    on="date",
    by="ticker",
    direction="backward",
    allow_exact_matches=True
).rename(columns={"cum_news":"cum_curr"})

prev_left = (
    shorts[["ticker","prev_date"]]
    .dropna(subset=["prev_date"])
    .rename(columns={"prev_date":"date_left"})
    .sort_values("date_left", kind="mergesort")
    .reset_index(drop=True)
)

right_prev = news[["ticker","date","cum_news"]].sort_values("date", kind="mergesort").reset_index(drop=True)

prev = pd.merge_asof(
    left=prev_left,
    right=right_prev,
    left_on="date_left",
    right_on="date",
    by="ticker",
    direction="backward",
    allow_exact_matches=True
).rename(columns={"cum_news":"cum_prev"})[["ticker","date_left","cum_prev"]]

news_window = (
    curr
    .merge(shorts[["ticker","date","prev_date"]], on=["ticker","date"], how="left")
    .merge(prev, left_on=["ticker","prev_date"], right_on=["ticker","date_left"], how="left")
    .assign(
        cum_curr=lambda d: d["cum_curr"].fillna(0),
        cum_prev=lambda d: d["cum_prev"].fillna(0),
        news_since_prev_short=lambda d: (d["cum_curr"] - d["cum_prev"]).astype("int64")
    )[["ticker","date","news_since_prev_short"]]
    .sort_values(["ticker","date"], kind="mergesort")
    .reset_index(drop=True)
)

In [10]:
#prendo il numero di wiki views alla data degli shorts (calcolati dallo shorts precedente allo shorts attuale)

wiki["date"] = pd.to_datetime(wiki["date"], errors="coerce", utc=True).dt.tz_convert(None).dt.floor("D")
wiki = wiki.dropna(subset=["ticker","date"]).reset_index(drop=True)
wiki = wiki.sort_values(["ticker","date"], kind="mergesort").reset_index(drop=True)

wiki["wiki_views"] = pd.to_numeric(wiki["wiki_views"], errors="coerce").fillna(0)
wiki["cum_wiki"]   = wiki.groupby("ticker")["wiki_views"].cumsum()

left_curr  = shorts[["ticker","date"]].sort_values("date", kind="mergesort").reset_index(drop=True)
right_curr = wiki[["ticker","date","cum_wiki"]].sort_values("date", kind="mergesort").reset_index(drop=True)

curr = pd.merge_asof(
    left=left_curr,
    right=right_curr,
    on="date",
    by="ticker",
    direction="backward",
    allow_exact_matches=True
).rename(columns={"cum_wiki":"cum_curr"})

prev_left = (
    shorts[["ticker","prev_date"]]
    .dropna(subset=["prev_date"])
    .rename(columns={"prev_date":"date_left"})
    .sort_values("date_left", kind="mergesort")
    .reset_index(drop=True)
)

right_prev = wiki[["ticker","date","cum_wiki"]].sort_values("date", kind="mergesort").reset_index(drop=True)

prev = pd.merge_asof(
    left=prev_left,
    right=right_prev,
    left_on="date_left",
    right_on="date",
    by="ticker",
    direction="backward",
    allow_exact_matches=True
).rename(columns={"cum_wiki":"cum_prev"})[["ticker","date_left","cum_prev"]]

wiki_window = (
    curr
    .merge(shorts[["ticker","date","prev_date"]], on=["ticker","date"], how="left")
    .merge(prev, left_on=["ticker","prev_date"], right_on=["ticker","date_left"], how="left")
    .assign(
        cum_curr=lambda d: d["cum_curr"].fillna(0),
        cum_prev=lambda d: d["cum_prev"].fillna(0),
        wiki_views_since_prev_short=lambda d: (d["cum_curr"] - d["cum_prev"]).astype("int64")
    )[["ticker","date","wiki_views_since_prev_short"]]
    .sort_values(["ticker","date"], kind="mergesort")
    .reset_index(drop=True)
)

In [12]:
#unisco price con shorts
df_merged = (
    prices_on_short
      .merge(shorts.rename(columns={"volume": "si_volume"}),
             on=["ticker","date"], how="left")
      .sort_values(["ticker","date"])
      .reset_index(drop=True)
)

In [13]:
#unisco trends
df_merged = (
    df_merged.merge(trends_window, on=["ticker","date"], how="left")
        .assign(trend_score2w=lambda d: d["trend_score2w"].fillna(0))
        .sort_values(["ticker","date"])
        .reset_index(drop=True)
)

In [14]:
#unisco news
df_merged = (
    df_merged.merge(news_window, on=["ticker","date"], how="left")
         .assign(news_since_prev_short=lambda d: d["news_since_prev_short"].fillna(0).astype("int64"))
         .sort_values(["ticker","date"])
         .reset_index(drop=True)
)

In [15]:
#unisco wiki_views
df_merged = (
    df_merged.merge(wiki_window, on=["ticker","date"], how="left")
         .assign(wiki_views_since_prev_short=lambda d: d["wiki_views_since_prev_short"].fillna(0).astype("int64"))
         .sort_values(["ticker","date"])
         .reset_index(drop=True)
)

In [16]:
print( len(df_merged["ticker"].unique().tolist()) )

50


In [18]:
#sistemo df finale
df_merged = df_merged.drop(columns=["prev_date"])
df_merged = df_merged.dropna().sort_values(["ticker", "date"]).reset_index(drop=True)
df_merged = df_merged[["date","ticker", *[c for c in df_merged.columns if c not in ("date","ticker")]]]
df_merged.columns = ["date", "ticker", "close", "d2c", "shorts", "volume", "trend_score", "news_volume", "wiki_views"]


In [19]:
df_merged

,date,ticker,close,d2c,shorts,volume,trend_score,news_volume,wiki_views
0,2020-10-30,AAL,11.2800,3.14,172765872,55107304,15.5,12,7010
1,2020-11-13,AAL,12.2400,1.93,141817804,73589497,15.5,21,5491
2,2020-11-30,AAL,14.1300,1.76,141848467,80739188,20.0,41,7820
3,2020-12-15,AAL,17.0100,1.59,160823723,101002470,22.5,54,6571
4,2020-12-31,AAL,15.7700,3.13,171336981,54682584,16.0,48,7490
...,...,...,...,...,...,...,...,...,...
4913,2024-12-31,ZOM,0.1200,7.21,45377632,6293854,1.0,2,0
4914,2025-01-15,ZOM,0.1300,3.95,44812240,11337256,1.0,1,0
4915,2025-01-31,ZOM,0.1229,11.91,44586633,3742116,1.0,0,0
4916,2025-02-14,ZOM,0.1207,14.20,45403440,3197598,1.0,2,0


In [20]:
df_merged.to_csv( f"./data/{INDEX}/merged_data.csv", index=False )